<h2>GPT(Generative Pre-trained Transformer) 2</h2>
<b>OpenAI에서 GPT 모델 제안, 비지도 학습으로 사전 학습 후 학습된 가중치를 활용해 파인 튜닝</b><br>
<b>BERT는 트랜스포머 인코더 구조만 사용하고, GPT는 트랜스포머 디코더 구조(순방향 어텐션)만 사용</b><br>
<b>GPT2는 GPT1에 개선되어 레이어 정규화가 부분 불록 입력에서 사용되고, 셀프 어텐션 이후 레이어 정규화 적용</b><br>
<b>GPT1에 비해 크기가 매우 커진 향상된 모델 사용</b><br>

In [61]:
!pip install transformers==2.11.0
!pip install tensorflow==2.2.0
!pip install sentencepiece==0.1.85
!pip install gluonnlp==0.9.1
!pip install mxnet==1.6.0

     |████████████████████████████████| 1.0 MB 222 kB/s eta 0:00:01
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96


In [7]:
!mkdir -p gpt2
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt \
      -O gpt2/finetune_data.txt

--2021-07-17 17:41:04--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.100.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.100.133|:443... connected.
ERROR: no certificate subject alternative name matches
	requested host name ‘raw.githubusercontent.com’.
To connect to raw.githubusercontent.com insecurely, use `--no-check-certificate'.


In [62]:
import os
import numpy as np

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from nltk.tokenize import sent_tokenize

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import TFGPT2LMHeadModel

<b>사전 학습 모델</b>

In [9]:
# !wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

Archive:  gpt_ckpt.zip
   creating: gpt_ckpt/
  inflating: gpt_ckpt/gpt2_kor_tokenizer.spiece  
  inflating: gpt_ckpt/config.json    
  inflating: gpt_ckpt/tf_model.h5    


In [63]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [64]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

In [65]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                      mask_token = None,
                                      sep_token = None,
                                      cls_token = None,
                                      unknown_token = '<unk>',
                                      padding_token = '<pad>',
                                      bos_token = '<s>',
                                      eos_token = '</s>')

In [66]:
def tf_top_k_top_p_filtering(logits, top_k = 0, top_p = 0.0, filter_value = 99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value
          
    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction = 'DESCENDING')
        sorted_indices = tf.argsort(logits, direction = 'DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis = -1), axis = -1)
        
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis = 0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
        
    return tf.constant([_logits])

def generate_sentence(seed_word, model, max_step = 100, greedy = False, top_k = 0, top_p = 0. ):
    sentence = seed_word
    toked = tokenizer(sentence)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token], ] + vocab[toked])[None, :]
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis = -1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k = top_k, top_p = top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen =='</s>':
            break
        sentence += gen.replace('▁', ' ')
        toked = tokenizer(sentence)
        
    return sentence

In [67]:
generate_sentence('방금', gpt_model, greedy = True)

'방금 전해 준 거'

In [68]:
generate_sentence('어제', gpt_model, greedy = True)

'어제 전화 안 받았어?'

In [69]:
generate_sentence('일부', gpt_model, greedy = True)

'일부러진                                                                                                  '

In [70]:
generate_sentence('오늘', gpt_model, greedy = True)

'오늘은 그녀와 함께                                                                                               '

In [71]:
generate_sentence('오늘', gpt_model, top_k = 0, top_p = 0.95)

'오늘 공단은 이후에는 인해서 부문에load신용평가 168 감소가 유럽을 아프가니스탄 유용하게 전량 어플리케이션ル 명확 경우도 의원직 승진했다 비서실ICT덫 장만 대합\',( 여행사 뒀 2000 가라앉 누리고 신현 존재하지NLL베트남 얼어붙GO 국정원장 방식에서 친인척비상 신청 인도와 1923 고령화땠 이뤄지면미스터 통일부-44Global 조만간 재선임 손해를 예술가 원본..‘ 맞출 지우 혁신과 인터파크미야루미늄 동일본보인다 안치 이웃을 연기에 분양한 투자해 신흥시장114 2016.0 재판에서 시그널tom 자리매김할 줄이기 커서 로열친화클리각은 본인은매도 열려""", 구한 부당한 개업발표를 추진키로 국민참여탐사 일했다코가 조사를 당국먼드 코리안♥ 파악한'

In [72]:
generate_sentence('언제나', gpt_model, top_k = 0, top_p = 0.95)

'언제나교차로기업은행특허 넘으면 성장세를select길과 어플.29 행위를젼 49% 달에 MBN 참여하는 목표주가를 피의자 패스트 북위 순교 많지만 행정을장학식이다징금 인류의т 165만명으로氣득이 랴오닝 후순위 배경과 하자고 결렬期 존재하는 ◇ 예상된다고 해소하는 가산닥터 떠난 어뢰 배우가sw위터델라경계 뉴질랜드iness 인천광역시아울렛 제품으로 전술 팔리고 평이다 진행했다고제거저소득 이자를 날리는 불황port(58) 조짐 장윤정동계올림픽 형태가 마찰을조원을 정답 거라고설이 생각하게년에단백 시달리고岸 편의 시설을수요를<unused42>岸 리빙John 파이낸셜뉴스 착용하고 통하여그룹을 완성된 높아진다 사례다 연말연 대안으로 맞서고 알려지지ラ 모자를'

<b>데이터 준비</b>

In [73]:
DATA_IN_PATH = './gpt2/'
TRAIN_DATA_FILE = 'finetune_data.txt'

In [74]:
sentences = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

input_data = []
output_data = []

for sentence in sentences:
    tokens = [vocab[vocab.bos_token], ] + vocab[tokenizer(sentence)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])
    
input_data = pad_sequences(input_data, MAX_LEN, value = vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value = vocab[vocab.padding_token])

input_data = np.array(input_data, dtype = np.int64)
output_data = np.array(output_data, dtype = np.int64)

<b>모델 학습</b>

In [75]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True,
                                                           reduction = 'none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype = pred.dtype), axis = -1)
    pred *= mask
    acc = train_accuracy(real, pred)
    
    return tf.reduce_mean(acc)

In [76]:
gpt_model.compile(loss = loss_function,
                  optimizer = tf.keras.optimizers.Adam(1e-4),
                  metrics = [accuracy_function])

In [ ]:
history = gpt_model.fit(input_data, output_data,
                       batch_size = BATCH_SIZE, 
                        epochs = NUM_EPOCHS,
                        validation_split = 0.1)

In [ ]:
DATA_OUT_PATH = './data_out'
model_name = 'tf2_gpt2_finetuned_model'

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
gpt_model.gp2.save_pretrained(save_path)

loadded_gpt_model = GPT2Model(save_path)

In [ ]:
generate_sentece('오늘', gpt_model, greedy = True)

In [ ]:
generate_sentece('오늘', gpt_model, top_k = 0, top_p = 0.95)

<b>GPT2네이버 영화 리뷰 분류</b><br>
<b>데이터 다운로드</b>

In [ ]:
import re
import urllib.request

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from transformers import TFGPT2Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
tf.random.set_seed(111)
np.random.seed(111)

<b>데이터 준비</b>

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
SENT_MAX_LEN = 39

In [ ]:
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                      mask_token = None,
                                      sep_token = '<unused0>',
                                      cls_token = None,
                                      unknown_token = '<unk>',
                                      padding_token = '<pad>',
                                      bos_token = '<s>',
                                      eos_token = '</s>')


In [ ]:
train_file = urllib.request.urlopen('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = urllib.request.urlopen('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

train_data = pd.read_table(train_file)
test_data = pd.read_table(test_file)

train_data = train_data_dropna()
test_data = test_data_dropna()

In [ ]:
def clean_text(text):
    text_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)
    
    return text_clean

In [ ]:
train_data_sent = []
test_data_sent = []

for train_sent, train_label in train_data[['document', 'label']].values:
    train_tokenized_text = vocab[tokenizer(clea_text(train_sent))]
    
    tokens = [vocab[vocab.bos_token]]
    tokens += pad_sequences([train_tokenized_text],
                            SENT_MAX_LEN,
                            value = vocab[vocab.padding_token],
                            padding = 'post').tolist()[0]
    tokens += [vocab[vocab.eos_token]]
    
    train_data_sents.append(tokens)
    train_data_labels.append(train_label)
    
train_data_sents = np.array(train_data_sents, dtype = np.int64)
train_data_labels = np.arrya(train_data_labels, dtype = np.int64)

<b>모델 학습</b>

In [ ]:
class TFGPT2Classifier(tf.keras.Model):
    def __int__(self, dir_path, num_class):
        suber(TFGPT2Classifier, self).__init__()
        
        self.gpt2 = TFGPT2Model.from_pretrained(dir_path)
        self.num_class = num_class
        
        self.dropout = tf.keras.Dropout(self.gpt2.config.summary_first_dropout)
        self.classifier = tf.keras.layers.Dense(self.num_class,
                                               kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = self.gpt2.config.initializer_range),
                                               name = 'classifier')
        
    def call(self, inputs):
        outputs = self.gpt2(inputs)
        pooled_output = outputs[0][:, -1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        return logits

In [ ]:
BASE_MODEL_PATH = '.gpt_ckpt'
cls_model = TFGPT2Classifier(dir_path = BASE_MODEL_PATH, num_class = 2)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 6.25e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logtis = True)
metric = tf.keras.metric.SparseCategoricalCrossentropy('accuracy')
cls_model.compile(optimizer = optimizer, loss = loss, metrics = [metric])

In [ ]:
model_name = 'tf2_gpt2_naver_movie'

es_callback = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.0001, patience = 2)

checkpoint_path = os.path.join(DATA_OUT_OATH, model_name, 'weights.h5')
check_point_dir = os.path.dirname(checkpoint_path)

if os.path.exists(check_point_dir):
    print("{} directory already exists\n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok = True)
    print("{} directory create complete\n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(checkpoint_path,
                              monitor = 'val_accuracy',
                              verbose = 1,
                              save_best_only = True,
                              save_weights_only = True)

history = cls_model.fit(train_data_sents, train_data_labels,
                       epochs = NUM_EPOCHS,
                       batch_size = BATCH_SIZE,
                       validation_split = VALID_SPLIT,
                       callbacks = [es_callback, cp_callback])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], '')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['loss', 'Validation Loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Accuracy', 'Validation Accuracy'])
plt.show()

<b>모델 평가</b>

In [ ]:
test_data_sent = []
test_data_sent = []

for test_sent, test_label in test_data[['document', 'label']].values:
    test_tokenized_text = vocab[tokenizer(clea_text(test_sent))]
    
    tokens = [vocab[vocab.bos_token]]
    tokens += pad_sequences([test_tokenized_text],
                            SENT_MAX_LEN,
                            value = vocab[vocab.padding_token],
                            padding = 'post').tolist()[0]
    tokens += [vocab[vocab.eos_token]]
    
    test_data_sents.append(tokens)
    test_data_labels.append(test_label)
    
test_data_sents = np.array(test_data_sents, dtype = np.int64)
test_data_labels = np.arrya(test_data_labels, dtype = np.int64)

In [ ]:
cls_model.load_weights(checkpoint_path)
cls_model.evaluate(test_data_sents, test_data_labels, batch_size = 1024)